In [1]:
import os
import sys
from functools import partial

import numpy as np
import tensorflow as tf
from scipy.stats import beta

sys.path.append(os.path.join("..", "..", ".."))
import cardio.dataset as ds
from cardio import EcgDataset
from cardio.dataset import B, V, F
from cardio.models.dirichlet_model import DirichletModel, concatenate_ecg_batch
from cardio.models.metrics import f1_score, classification_report, confusion_matrix

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/anaconda3/envs/tensorflow/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
Using TensorFlow backend.


In [2]:
MODEL_PATH = os.path.expanduser("~")+"/coding/cnn/DeepECG/dirichlet_model"

BATCH_SIZE = 100

In [3]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5, allow_growth=True)

model_config = {
    "session": {"config": tf.ConfigProto(gpu_options=gpu_options)},
    "build": False,
    "load": {"path": MODEL_PATH},
}

In [4]:
template_predict_ppl = (
    ds.Pipeline()
      .init_model("static", DirichletModel, name="dirichlet", config=model_config)
      .init_variable("predictions_list", init_on_each_run=list)
      .load(fmt="wfdb", components=["signal", "meta"])
      .flip_signals()
      .split_signals(2048, 2048)
      .predict_model("dirichlet", make_data=partial(concatenate_ecg_batch, return_targets=False),
                     fetches="predictions", save_to=V("predictions_list"), mode="e")
      .run(batch_size=BATCH_SIZE, shuffle=False, drop_last=False, n_epochs=1, lazy=True)
)

INFO:tensorflow:Restoring parameters from /Users/truongnm/coding/cnn/DeepECG/dirichlet_model/model-26000


Original file

In [5]:
signal_path = os.path.expanduser("~")+"/coding/cnn/DeepECG/raw/A00001.mat"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [6]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021257801, 'NO': 0.97874224},
  'uncertainty': 0.007573604583740234}]

MAT -> csv: `python mat2csv.py "raw/A00001.mat"`

-> full signal: `wrsamp -i raw/A00001.csv -o raw/A00001-converted -G 1000 -F 300 -z`

-> MAT file: `wfdb2mat -r raw/A00001-converted` and move output files to raw folder

In [7]:
signal_path = os.path.expanduser("~")+"/coding/cnn/DeepECG/raw/A00001-convertedm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [8]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021257801, 'NO': 0.97874224},
  'uncertainty': 0.007573604583740234}]

DB MAT file -> csv -> full signal

-> signal: `wrsamp -i raw/A00001.csv -o raw/A00001-converted-1s -G 1000 -F 300 -z -t 250`

-> MAT file: `wfdb2mat -r raw/A00001-converted-1s` and move output files to raw folder

-> test: 

In [10]:
signal_path = os.path.expanduser("~")+"/coding/cnn/DeepECG/raw/A00001-converted-1sm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [11]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021675928, 'NO': 0.97832406},
  'uncertainty': 0.0073926448822021484}]

DB MAT file -> csv -> full signal (same above)

-> csv 250 lines (1s): `wrsamp -i raw/A00001.csv -o raw/A00001-converted-1s -G 1000 -F 300 -z -t 250`

-> gnuplot image - this can be image received from user: 
```
gnuplot -e "fileIn='raw/A00001-converted-1s.csv'; fileOut='raw/A00001-converted-1s.png'" csv2img.gnuplot
```

-> csv: `python img2csv.py "/Users/truongnm/coding/cnn/ecg/ecg-process/raw/A00001-converted-1s.png"`

-> signal: `wrsamp -i raw/A00001-converted-1s-from-user.csv -o raw/A00001-converted-1s-from-user -G 1000 -F 300 -z`

-> MAT: `wfdb2mat -r raw/A00001-converted-1s-from-user`

-> test:

In [12]:
signal_path = os.path.expanduser("~")+"/coding/cnn/DeepECG/raw/A00001-converted-1s-from-userm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [13]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021675965, 'NO': 0.978324},
  'uncertainty': 0.00739288330078125}]

### Test with another patient

Original file

In [14]:
signal_path = os.path.expanduser("~")+"/coding/cnn/DeepECG/raw/A00004.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [15]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.71521246, 'NO': 0.2847875},
  'uncertainty': 0.5845246315002441}]

MAT -> csv: `python mat2csv.py "raw/A00004.mat"`

-> full signal: `wrsamp -i raw/A00004.csv -o raw/A00004-converted -G 1000 -F 300 -z`

-> MAT file: `wfdb2mat -r raw/A00004-converted` and move output files to raw folder

In [16]:
signal_path = os.path.expanduser("~")+"/coding/cnn/DeepECG/raw/A00004-convertedm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [17]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.71521246, 'NO': 0.2847875},
  'uncertainty': 0.5845246315002441}]

DB MAT file -> csv -> full signal (same above)

-> signal for 1s: `wrsamp -i raw/A00004.csv -o raw/A00004-converted-1s -G 1000 -F 300 -z -t 250`

-> MAT file: `wfdb2mat -r raw/A00004-converted-1s` and move output files to raw folder

-> test: 

In [18]:
signal_path = os.path.expanduser("~")+"/coding/cnn/DeepECG/raw/A00004-converted-1sm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [19]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.02360226, 'NO': 0.97639775},
  'uncertainty': 0.008587837219238281}]

If the input was 30 seconds, this patient got AFib. But if the input reduced to 1s, our network consider this patient is normal. Or at least from 0s to 1s, nothing abnormal in ECG.

DB MAT file -> csv -> full signal (same above)

-> signal for 1s: `wrsamp -i raw/A00004.csv -o raw/A00004-converted-1s -G 1000 -F 300 -z -t 250`

-> csv: `rdsamp -r raw/A00004-converted-1s -c -pd > raw/A00004-converted-1s.csv`

-> gnuplot image - this can be image received from user: 
```
gnuplot -e "fileIn='raw/A00004-converted-1s.csv'; fileOut='raw/A00004-converted-1s.png'" csv2img.gnuplot
```

-> csv: `python img2csv.py "/Users/truongnm/coding/cnn/ecg/ecg-process/raw/A00004-converted-1s.png"`

-> signal: `wrsamp -i raw/A00004-converted-1s-from-user.csv -o raw/A00004-converted-1s-from-user -G 1000 -F 300 -z`

-> MAT: `wfdb2mat -r raw/A00004-converted-1s-from-user`

-> test:

In [20]:
signal_path = os.path.expanduser("~")+"/coding/cnn/DeepECG/raw/A00004-converted-1s-from-userm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [21]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021675685, 'NO': 0.9783243},
  'uncertainty': 0.0073926448822021484}]

Attempt to predict from image which generated from full csv:
-> full csv: `rdsamp -r raw/A00004-converted -c -pd > raw/A00004-converted.csv`

-> Gnuplot:
```
gnuplot -e "fileIn='raw/A00004-converted.csv'; fileOut='raw/A00004.png'" csv2img.gnuplot
```
-> csv: `python img2csv.py "/Users/truongnm/coding/cnn/ecg/ecg-process/raw/A00004.png"`

-> signal: `wrsamp -i raw/A00004-from-user.csv -o raw/A00004-from-user -G 1000 -F 300 -z`

-> MAT: `wfdb2mat -r raw/A00004-from-user`

-> test:


In [22]:
signal_path = os.path.expanduser("~")+"/coding/cnn/DeepECG/raw/A00004-from-userm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [23]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021675652, 'NO': 0.97832435},
  'uncertainty': 0.007392406463623047}]